Install and load all dependencies (first time only) \
NOTE: you may need to restart the runtime afterwards (CTRL+M .).

In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py

Set up the custom Hopper environment



1.   Upload `classes.zip` to the current session's file storage
2.   Un-zip it by running cell below


In [ ]:
!unzip classes.zip



---



\

**Test an RL agent on the OpenAI Gym Hopper environment**

\

In [ ]:
import torch
import gym

from env.custom_hopper import *
from agent import Agent, Policy

In [2]:
model = None # Fill in model path
device = 'cpu'
episodes = 100

In [ ]:
env = gym.make('CustomHopper-source-v0')
# env = gym.make('CustomHopper-target-v0')

print('Action space:', env.action_space)
print('State space:', env.observation_space)
print('Dynamics parameters:', env.get_parameters())

observation_space_dim = env.observation_space.shape[-1]
action_space_dim = env.action_space.shape[-1]

policy = Policy(observation_space_dim, action_space_dim)
policy.load_state_dict(torch.load(model), strict=True)

agent = Agent(policy, device=device)

In [ ]:
total_reward = 0
for episode in range(episodes):
  done = False
  test_reward = 0
  state = env.reset()

  while not done:
    action, _ = agent.get_action(state, evaluation=True)
    
    state, reward, done, info = env.step(action.detach().cpu().numpy())

    test_reward += reward

  print(f"Episode: {episode} | Return: {test_reward}")
  total_reward += test_reward
print(f"average reward: {total_reward / episodes}")